# Last step
# Predict segmentations

In [3]:
%matplotlib inline

In [4]:
import os
import os.path

import glob

import matplotlib.pyplot as plt
import numpy as np

import skimage.io
import skimage.morphology

import tqdm

import tensorflow as tf
import keras

import utils.metrics
import utils.model_builder

Configurable parameter:
* config (experiment)
* GPU
* input folder 
* output folder

# Configuration

In [5]:
GPU_NO = "0"
#input_directory =  "/home/ubuntu/data/20190430/input"
input_directory = "/home/ubuntu/hudy_data/Pre_processing"
output_directory = "/home/ubuntu/hudy_data/output"
experiment_name = "neutros_v3"

# Initialize keras 

In [6]:
from config import config_vars
config_vars["root_directory"] = '/home/ubuntu/data/unet/'
print(config_vars)

# load configuration file
config_vars = utils.dirtools.setup_experiment(config_vars, experiment_name)

# output directory
config_vars["probmap_out_dir"] = output_directory



{'root_directory': '/home/ubuntu/data/unet/', 'max_training_images': 0, 'create_split_files': True, 'training_fraction': 0.5, 'validation_fraction': 0.25, 'transform_images_to_PNG': True, 'pixel_depth': 8, 'min_nucleus_size': 40, 'boundary_size': 2, 'augment_images': True, 'elastic_points': 16, 'elastic_distortion': 5, 'elastic_augmentations': 10, 'learning_rate': 1e-05, 'epochs': 15, 'steps_per_epoch': 500, 'batch_size': 10, 'val_batch_size': 10, 'rescale_labels': True, 'crop_size': 256, 'cell_min_size': 40, 'boundary_boost_factor': 1, 'object_dilation': 3, 'raw_images_dir': '/storage/data/2018_tim_tracking/unet/raw_images/', 'raw_annotations_dir': '/storage/data/2018_tim_tracking/unet/raw_annotations/', 'path_files_training': '/storage/data/2018_tim_tracking/unet/training.txt', 'path_files_validation': '/storage/data/2018_tim_tracking/unet/validation.txt', 'path_files_test': '/storage/data/2018_tim_tracking/unet/test.txt', 'normalized_images_dir': '/storage/data/2018_tim_tracking/une

In [7]:

# initialize GPU
configuration = tf.ConfigProto()
configuration.gpu_options.allow_growth = True
configuration.gpu_options.visible_device_list = GPU_NO

session = tf.Session(config = configuration)

# apply session
keras.backend.set_session(session)


# Load and preprocessing images

In [8]:
def load_and_prepare_images_from_directory(input_directory):
    image_names = glob.glob(input_directory + "/*png")
    images = []
    imagebuffer = skimage.io.imread_collection( image_names )
    for image in imagebuffer:
        #image = skimage.io.imread(filename)
        images.append(skimage.color.rgb2gray(image))

    print(input_directory)
    print("found {} images".format(len(imagebuffer)))

    images = np.array(images)
    
    
    dim1 = np.floor(images.shape[1]/16) * 16 
    dim1 = dim1.astype(np.int)
    dim2 = np.floor(images.shape[2]/16) * 16 
    dim2 = dim2.astype(np.int)

    images = images[:,0:dim1,0:dim2]

    dim1 = images.shape[1]
    dim2 = images.shape[2]

    images = images.reshape((-1, dim1, dim2, 1))

    #print(dim1,dim2)

    # preprocess images
    percentile = 99.9
    for image_no in range(images.shape[0]):
        orig_img = images[image_no,:,:,:]

        high = np.percentile(orig_img, percentile)
        low = np.percentile(orig_img, 100-percentile)

        img = np.minimum(high, orig_img)
        img = np.maximum(low, img)
        img = (img - low) / (high - low) 
        img = skimage.img_as_ubyte(img) 
        images[image_no,:,:,:] = img # gives float64, thus cast to 8 bit later



    images = images.astype(float)
    images = images / 256

    return(images,imagebuffer)

# Predict images 

In [ ]:
for directory in tqdm.tqdm(os.listdir(input_directory)):
    
    if os.path.exists(os.path.join(output_directory,directory)):
        print("Folder {} processed!".format(directory))
    else:
        
    
        [images,imagebuffer] = load_and_prepare_images_from_directory(os.path.join(input_directory,directory))

        # build model and load weights
        dim1 = images.shape[1]
        dim2 = images.shape[2]
        model = utils.model_builder.get_model_3_class(dim1, dim2)
        model.load_weights(config_vars["model_file"])

        #  prediction 
        predictions = model.predict(images, batch_size=1)


        os.makedirs(os.path.join(output_directory,directory)) 
        for i in range(len(images)):

            image_savename = os.path.join(
                output_directory, 
                directory, 
                os.path.basename(imagebuffer.files[i])
            )

            probmap = predictions[i].squeeze()

            skimage.io.imsave(os.path.splitext(image_savename)[0] + ".png", probmap)

    
    

  0%|          | 0/60 [00:00<?, ?it/s]

Folder drugs_0705_001.nd2 - drugs_0705_001.nd2 (series 09) processed!
Folder wash_1245_add_dox001.nd2 - wash_1245_add_dox001.nd2 (series 11) processed!
Folder before002.nd2 - before002.nd2 (series 04) processed!
Folder drugs_0705_001.nd2 - drugs_0705_001.nd2 (series 01) processed!
Folder drugs_0705_001.nd2 - drugs_0705_001.nd2 (series 02) processed!
Folder wash_1245_add_dox001.nd2 - wash_1245_add_dox001.nd2 (series 04) processed!
Folder drugs_0705_001.nd2 - drugs_0705_001.nd2 (series 06) processed!
Folder before002.nd2 - before002.nd2 (series 05) processed!
Folder before002.nd2 - before002.nd2 (series 12) processed!
Folder before002.nd2 - before002.nd2 (series 08) processed!
Folder before002.nd2 - before002.nd2 (series 14) processed!
Folder drugs_0705_001.nd2 - drugs_0705_001.nd2 (series 08) processed!
Folder wash_1245_add_dox001.nd2 - wash_1245_add_dox001.nd2 (series 18) processed!
Folder wash_1245_add_dox001.nd2 - wash_1245_add_dox001.nd2 (series 15) processed!
Folder wash_1245_add_d

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


Instructions for updating:
Colocations handled automatically by placer.


/home/ubuntu/repos/unet4neutrophils/utils/model_builder.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  a = keras.layers.Convolution2D(64, 3, 3, **option_dict_conv)(x)
/home/ubuntu/repos/unet4neutrophils/utils/model_builder.py:17: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9)`
  a = keras.layers.BatchNormalization(**option_dict_bn)(a)
/home/ubuntu/repos/unet4neutrophils/utils/model_builder.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  a = keras.layers.Convolution2D(64, 3, 3, **option_dict_conv)(a)
/home/ubuntu/repos/unet4neutrophils/utils/model_builder.py:20: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9)`
  a = keras.layers.BatchNormalization(**option_dict_bn)(a)
/home/ubuntu/repos/unet4neutrophils/utils/model_builder.

...

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/hudy_data/output/before002.nd2 - before002.nd2 (series 17)/before002_s17_f2_t0073.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/hudy_data/output/before002.nd2 - before002.nd2 (series 17)/before002_s17_f2_t0046.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/hudy_data/output/before002.nd2 - before002.nd2 (series 17)/before002_s17_f1_t0057.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/hudy_data/output/before002.nd2 - before002.nd2 (series 

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/hudy_data/output/before002.nd2 - before002.nd2 (series 17)/before002_s17_f2_t0016.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/hudy_data/output/before002.nd2 - before002.nd2 (series 17)/before002_s17_f2_t0077.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/hudy_data/output/before002.nd2 - before002.nd2 (series 17)/before002_s17_f2_t0030.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/hudy_data/output/before002.nd2 - before002.nd2 (series 

/home/ubuntu/hudy_data/Pre_processing/before002.nd2 - before002.nd2 (series 09)
found 188 images


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/hudy_data/output/before002.nd2 - before002.nd2 (series 09)/before002_s09_f1_t0003.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/hudy_data/output/before002.nd2 - before002.nd2 (series 09)/before002_s09_f1_t0002.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/hudy_data/output/before002.nd2 - before002.nd2 (series 09)/before002_s09_f1_t0011.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/ubuntu/hudy_data/output/before002.nd2 - before002.nd2 (series 

/home/ubuntu/hudy_data/Pre_processing/wash_1245_add_dox001.nd2 - wash_1245_add_dox001.nd2 (series 13)
found 860 images


In [ ]:
print ((os.path.join(output_directory,directory)))
    